In [5]:
import pandas as pd 

from showupforhealth import params, utils
from showupforhealth.ml_functions import preprocessor

IndentationError: unindent does not match any outer indentation level (preprocessor.py, line 75)

In [ ]:
data = pd.read_csv(f'{OUTPUT_DATA}training_data_no_features.csv')

In [4]:
data.head

NameError: name 'data' is not defined